# BERT를 활용해서 Extraction-based MRC 문제 풀기

 * GPU 확인

In [ ]:
!nvidia-smi

* huggingFace 에서 필요한 library 설치

In [ ]:
!pip install datasets==1.4.1
!pip install transformers==4.4.1

 * huggingFace 에서 활용할 코드 파일을 git repo에서 가져오기

In [ ]:
# To use utility functions defined in examples.
!git clone https://github.com/huggingface/transformers.git
%cd transformers
!git checkout v4.4.2-release
%cd ..
import sys
sys.path.append('transformers/examples/question-answering')

## 데이터 및 평가 지표 불러오기

* 데이터셋 가져오기
  * KorQuAD 데이터셋 사용

In [ ]:
from datasets import load_dataset

datasets = load_dataset("squad_kor_v1")

* 데이터셋 확인
  * train과 validation으로 이루어져 있음(각각을 하나의 table로 생각하면 됨)
    * features(table의 header)

In [ ]:
datasets

In [ ]:
# access train data 
datasets['train']

In [ ]:
# access first data
datasets['train'][0]

* 데이터셋을 평가하는 function

In [ ]:
from datasets import load_metric

metric = load_metric('squad')

## Pre-trained 모델 불러오기

* AutoConfig : configuration을 불러올 수 있는 class
* AutoModelForQuestionAnswering : model을 불러올 수 있는 class
* AutoTokenizer

In [ ]:
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer
)

* model 이름 정의

In [ ]:
model_name = "bert-base-multilingual-cased"

* model_name에 해당하는 config, tokenizer, model을 가져옴

In [ ]:
config = AutoConfig.from_pretrained(
    model_name
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True
)
model = AutoModelForQuestionAnswering.from_pretrained(
    model_name,
    config=config
)

* model 실제 아키텍처 확인

In [ ]:
model

## 설정

* max_seq_length
  * sequence length의 한계를 알고 있어야 이max_seq_length 내에서 model을 만들고, padding을 정의할 수 있음
* pad_to_max_length
  * True : max_seq_length 범위 내에서 남는 부분을 padding으로 채워줌
* doc_stride
  * 나눈 두 문서의 overlap 되는 sequence 길이
  * 문서가 너무 긴 경우 문서를 일부 겹치게 하여(overlap) 두 개로 나눔
  * 두 개의 문서로 question-answering system을 진행함
  * 각각 문서에서 답변을 구한 후 답변을 취합하여 확률이 더 높은 답변을 가져감
* max_train_sampels
  * 학습할 train 데이터 최대 개수
  * max_train_samples 이하의 개수만 학습함
  * 값이 작을수록 학습이 빨리 끝남
* max_val_sampes
  * validation 에 사용될 데이터 최대 개수
* preprocessing_num_workers
  * 활용할 cpu thread 개수(process 개수)
  * 많이 쓸수록 빨라지지만, hardware에 depend하기도 하고, 4 이상으로 필요가 없는 경우도 있음
* batch_size
  * 학습할 때 사용하는 mini batch size
* num_train_epochs
  * 학습데이터를 재활용하는 횟수
* n_best_size
* max_answer_length
  * 답변의 최대 길이
  * 너무 긴 답변이 나오지 않도록 최대 길이를 제한함

In [ ]:
max_seq_length = 384 # 질문과 컨텍스트, special token을 합한 문자열의 최대 길이
pad_to_max_length = True
doc_stride = 128 # 컨텍스트가 너무 길어서 나눴을 때 오버랩되는 시퀀스 길이
max_train_samples = 16
max_val_samples = 16
preprocessing_num_workers = 4
batch_size = 4
num_train_epochs = 2
n_best_size = 20
max_answer_length = 30

## 전처리하기

* dataset을 적절하게 processing하여 모델에 넣기 적합하게 만듬
  * input : dataset들의 각각의 row
  * output : BERT의 input 형태

In [ ]:
def prepare_train_features(examples):
    # 주어진 텍스트를 토크나이징 한다. 이 때 텍스트의 길이가 max_seq_length를 넘으면 stride만큼 슬라이딩하며 여러 개로 쪼갬.
    # 즉, 하나의 example에서 일부분이 겹치는 여러 sequence(feature)가 생길 수 있음.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",  # max_seq_length까지 truncate한다. pair의 두번째 파트(context)만 잘라냄.
        max_length=max_seq_length,
        stride=doc_stride,
        return_overflowing_tokens=True, # 길이를 넘어가는 토큰들을 반환할 것인지
        return_offsets_mapping=True,  # 각 토큰에 대해 (char_start, char_end) 정보를 반환한 것인지
        padding="max_length",
    )
    
    # example 하나가 여러 sequence에 대응하는 경우를 위해 매핑이 필요함.
    overflow_to_sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # offset_mappings으로 토큰이 원본 context 내 몇번째 글자부터 몇번째 글자까지 해당하는지 알 수 있음.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # 정답지를 만들기 위한 리스트
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        
        # 해당 example에 해당하는 sequence를 찾음.
        sequence_ids = tokenized_examples.sequence_ids(i)
        
        # sequence가 속하는 example을 찾는다.
        example_index = overflow_to_sample_mapping[i]
        answers = examples["answers"][example_index]
        
        # 텍스트에서 answer의 시작점, 끝점
        answer_start_offset = answers["answer_start"][0]
        answer_end_offset = answer_start_offset + len(answers["text"][0])

        # 텍스트에서 현재 span의 시작 토큰 인덱스
        token_start_index = 0
        while sequence_ids[token_start_index] != 1:
            token_start_index += 1
        
        # 텍스트에서 현재 span 끝 토큰 인덱스
        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] != 1:
            token_end_index -= 1

        # answer가 현재 span을 벗어났는지 체크
        if not (offsets[token_start_index][0] <= answer_start_offset and offsets[token_end_index][1] >= answer_end_offset):
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # token_start_index와 token_end_index를 answer의 시작점과 끝점으로 옮김
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= answer_start_offset:
                token_start_index += 1
            tokenized_examples["start_positions"].append(token_start_index - 1)
            while offsets[token_end_index][1] >= answer_end_offset:
                token_end_index -= 1
            tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
train_dataset = datasets["train"]

* 실제 활용할 data를 부분적으로 고름
  * max_train_samples 개수만큼 가져오기

In [ ]:
train_dataset = train_dataset.select(range(max_train_samples))

In [ ]:
train_dataset[0]

* dictionary 형태는 BERT의 input 형태가 아니기 때문에 pre-process를 활용하여 BERT input 형태로 변환
  * map : train dataset을 매핑함
    * prepare_train_features : 사전에 정의한 전처리 함수 사용
    * batched
      * True : 하나씩 보는 것이 아닌 여러개가 합쳐진 방식으로 보게 됨
    * num_proc
      * pre-processing을 pipeline화 시킴으로서 processing을 할 때 한번에 하는 것이 아니라 필요할 때마다 ondemand하는 방식을 통해 데이터를 효율적으로 다룸

In [ ]:
column_names = datasets["train"].column_names
train_dataset = train_dataset.map( # mapping함
            prepare_train_features, # 전처리 함수
            batched=True, # 하나씩 보는 것이 아닌 여러개가 합쳐진 방식으로 보게 됨
            num_proc=preprocessing_num_workers, 
            remove_columns=column_names,
            load_from_cache_file=True,
        )

* train dataset 확인
  * BERT가 이해할 수 있는 형태로 변형된 것 확인

In [ ]:
train_dataset[0]

* pre-process validation

In [ ]:
def prepare_validation_features(examples):
    tokenized_examples = tokenizer(
        examples['question'],
        examples['context'],
        truncation="only_second",
        max_length=max_seq_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples


In [ ]:
eval_examples = datasets["validation"]
eval_examples = eval_examples.select(range(max_val_samples))
eval_dataset = eval_examples.map(
            prepare_validation_features,
            batched=True,
            num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=True,
        )

## Fine-tuning 하기

* Fine-tuning
  * 위키피디아 같은 거대한 corpus를 미리 학습시켜놓은 model을 가져와서 추가적으로 학습

* default_data_collator
  * 학습할 때 여러 example들을 collact해주는 역할을 함
* TrainingArguments
  * configuration이나 batch size같은 argument들을 합쳐서 한번에 줄 수 있는 convinient한 function
* EvalPrediction
  * evaluate할 때 prediction을 쉽게 할 수 있도록 도와주는 function
* QuestionAnsweringTrainer
  * training할 때 사용
  * 편하게 학습 가능
* postprocess_qa_predictions
  * model에서 나온 답변을 postprocess해줘야함


* 앞에서 git repo에서 import 한 파일들에 `trainer_qa`와 `utils_qa`가 포함되어 있음

In [1]:
from transformers import default_data_collator, TrainingArguments, EvalPrediction
from trainer_qa import QuestionAnsweringTrainer
from utils_qa import postprocess_qa_predictions

ModuleNotFoundError: ignored

* 나온 답을 원하는 형태로 mapping해주는 함수 구현
  * model이 이해할 수 있는 형태 -> 사람이 이해할 수 있는 형태

In [ ]:
def post_processing_function(examples, features, predictions):
    # Post-processing: we match the start logits and end logits to answers in the original context.
    predictions = postprocess_qa_predictions(
        examples=examples,
        features=features,
        predictions=predictions,
        version_2_with_negative=False,
        n_best_size=n_best_size,
        max_answer_length=max_answer_length,
        null_score_diff_threshold=0.0,
        output_dir=training_args.output_dir,
        is_world_process_zero=trainer.is_world_process_zero(),
    )
    
    # Format the result to the format the metric expects.
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in predictions.items()]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
    return EvalPrediction(predictions=formatted_predictions, label_ids=references)

* 특정 prediction이 답변과 비교했을 때 얼마나 좋은지 확인하는 함수

In [ ]:
def compute_metrics(p: EvalPrediction):
    return metric.compute(predictions=p.predictions, references=p.label_ids)

* argument 정의
  * output_dir
    * 내보낼 directory 정의
  * do_train
    * 학습 진행 여부
    * False이면 학습하지 않음
  * do_eval
    * evaluation 진행 여부
    * False이면 evaluation하지 않음
  * learning_rate
  * per_device_train_batch_size
    * 학습할 때의 batch size
  * per_device_eval_batch_size
    * evaluation할 때의 batch size
  * num_train_epochs
    * 학습할 때 재사용 횟수
  * weight_decay

In [ ]:
training_args = TrainingArguments(
    output_dir="outputs",
    do_train=True, 
    do_eval=True, 
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
)

* 편하게 학습하기 위해 trainer 정의
  * model
    * model 정의
  * args
    * 학습할 때 필요한 arguments 정의
  * train_dataset
    * 학습에 사용되는 dataset 정의
    * pre-processing을 통해 model이 이해할 수 있도록 mapping된 dataset
  * eval_Dataset
    * evaluation에 사용되는 dataset 정의
    * pre-processing을 통해 model이 이해할 수 있도록 mapping된 dataset
  * eval_examples
    * evaluation할 때 사용할 example을 정의
  * tokenizer
    * tokenizer 정의
  * data_collator
    * collator 방식(어떻게 example들을 같이 붙여줄지에 대한 방법)
    * 대부분 `default_data_collator`를 사용함
  * post_process_function
    * model output을 사람이 이해하는 형태로 해석하는 방식
    * dataset이 아닌 function을 input으로 받음
    * 받은 function을 내부에서 활용하여 필요할 때 ondemand로 나온 답변을 사람이 이해할 수 있는 형태로 바꿔줌
  * compute_metrics
    * evaluation 진행 방식
    * function을 input으로 넣어줌

In [ ]:
trainer = QuestionAnsweringTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        eval_examples=datasets["validation"],
        tokenizer=tokenizer,
        data_collator=default_data_collator,
        post_process_function=post_processing_function,
        compute_metrics=compute_metrics,
    )

* 학습하기

In [ ]:
train_result = trainer.train()

* 학습확인
  * global_step
    * 현재 몇 번째 batch를 활용하고 있는가
    * 실습에서 12번만 학습하기 때문에 12가 출력됨(학습 끝나고 확인하기 때문에 12만 출력됨)
  * training_loss
    * 현재 step에서의 loss

In [ ]:
train_result

## 평가하기

In [ ]:
metrics = trainer.evaluate()

In [ ]:
metrics

## 학습된 모델 불러오기

In [ ]:
finetuned_model = AutoModelForQuestionAnswering.from_pretrained('sangrimlee/bert-base-multilingual-cased-korquad')

In [ ]:
finetuned_model = finetuned_model.eval()

In [ ]:
finetuned_trainer = QuestionAnsweringTrainer(
    model=finetuned_model,
    args=TrainingArguments(
        output_dir="finetuned_outputs",
        do_eval=True, 
        per_device_eval_batch_size=batch_size,
    ),
    train_dataset=None,
    eval_dataset=eval_dataset,
    eval_examples=datasets["validation"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    post_process_function=post_processing_function,
    compute_metrics=compute_metrics,
)

In [ ]:
finetuned_metrics = finetuned_trainer.evaluate()

In [ ]:
finetuned_metrics